# ページごとの閲覧数を取得

In [1]:
import json
import pickle
from collections import defaultdict

import requests
from tqdm.notebook import tqdm

In [2]:
def get_daily_views(page_name, start_date="20201201", end_date="20211130"):
    page_name = page_name.replace("/", "%2F")

    endpoint = "https://wikimedia.org/api/rest_v1"\
                        "/metrics/pageviews/per-article/ja.wikipedia.org/all-access/all-agents" \
                        f"/{page_name}/daily/{start_date}/{end_date}"

    res = requests.get(endpoint, headers={"User-Agent": "s@89.io"})
    res.raise_for_status()

    daily_views = {}
    for item in res.json()["items"]:
        date = item["timestamp"][:8]
        views = item["views"]
        daily_views[date] = views
    
    return daily_views

In [3]:
get_daily_views("大谷翔平", start_date="20211101", end_date="20211107")

{'20211101': 7533,
 '20211102': 6206,
 '20211103': 5481,
 '20211104': 6356,
 '20211105': 6461,
 '20211106': 6822,
 '20211107': 5358}

In [4]:
# アクセス数がゼロの日は含まれない
get_daily_views("中森三弥", start_date="20211101", end_date="20211107")

{'20211103': 1, '20211106': 1}

---

In [5]:
with open("../data/workspace/page_locations_clean.json") as fp:
    page_locations_clean = json.load(fp)
len(page_locations_clean)

16528

In [6]:
with open("../data/workspace/page_views.pickle", "rb") as fp:
    page_views = pickle.load(fp)

In [7]:
for page, locations in tqdm(page_locations_clean.items()):
    if page in page_views:
        continue
    page_views[page] = get_daily_views(page)

  0%|          | 0/16528 [00:00<?, ?it/s]

In [8]:
len(page_views)

16528

In [9]:
# JSONではファイルサイズが100MB以上と大きくなるため、pickleでシリアライズ
# GitHubレポジトリへアップロードするため（100MB制限）
with open("../data/workspace/page_views.pickle", "wb") as fp:
    pickle.dump(page_views, fp)

## 確認

In [10]:
import pandas as pd

In [11]:
df_count = pd.DataFrame([(k, len(v)) for k, v in page_views.items()], columns=["ページ", "view日数"]).sort_values("view日数")
df_count

,ページ,view日数
5262,天野将平,79
4809,大井成元_(サッカー),96
11159,清水達也_(野球),99
1603,中島愛_(声優),170
1810,中森三弥,170
...,...,...
6091,小林浩美,365
6092,小林涼,365
6094,小林珠維,365
6096,小林直己,365


### 「アクセスのない日」が存在するケース

- 誰もアクセスしなかった
- そのときにまだページが存在しなかった
- そのページがのちに削除された、移動（リネーム）された

In [12]:
df_count.query("view日数 < 365")

,ページ,view日数
5262,天野将平,79
4809,大井成元_(サッカー),96
11159,清水達也_(野球),99
1603,中島愛_(声優),170
1810,中森三弥,170
...,...,...
5838,小倉順平,364
2418,仲摩純平,364
1740,中村昌秀,364
11891,田向泰輝,364


In [13]:
print(
    "「アクセスのない日」が存在するページ: 全体の" +  \
    f"""{df_count.query("view日数 < 365").shape[0] / df_count.shape[0]:.1%}"""
)

「アクセスのない日」が存在するページ: 全体の20.4%


In [14]:
df_count.groupby("view日数").count()\
    .reset_index().rename(columns={"ページ": "該当ページ数"})\
    .sort_values("該当ページ数", ascending=False)

,view日数,該当ページ数
135,365,13154
134,364,626
133,363,334
132,362,220
131,361,186
...,...,...
30,256,1
29,255,1
26,252,1
24,250,1
